# Libraries

In [1]:
!pip install -q catboost shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 KB 45.2 MB/s eta 0:00:00


In [2]:
import os 
os.chdir("/content/drive/MyDrive/UmojaHack Africa 2023: Carbon Dioxide Prediction Challenge (BEGINNER)/Data")

In [3]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from catboost import MetricVisualizer
import os
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from collections import Counter

import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
%matplotlib inline

shap.initjs()

# Data

In [4]:
sorted(os.listdir())

['SampleSubmission.csv', 'Test.csv', 'Train.csv', 'catboost_info']

In [5]:
ss, test, train = [pd.read_csv(f"{f}") for f in sorted(os.listdir())[:3]]

# Basic EDA

In [6]:
ss.head(2)

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-23.53_27.47_2019_1,0
1,ID_-23.53_27.47_2019_2,0


In [7]:
train.head(2)

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle,emission
0,ID_-23.73_28.77_2019_1,-23.73,28.77,2019,1,0.000167,0.713172,0.000102,0.223929,24.939440,...,5566.222019,54826.032616,4606.209995,19.115282,0.353778,24.951948,49.053953,-89.638032,17.988874,86.05170
1,ID_-23.73_28.77_2019_2,-23.73,28.77,2019,2,-0.000155,0.812910,-0.000137,0.080281,16.624162,...,3711.511365,68491.753342,3034.461661,12.218664,0.244987,4.708856,42.586683,-89.904314,22.773246,88.87567


In [8]:
test.head(2)

,ID_LAT_LON_YEAR_WEEK,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,ID_-23.53_27.47_2019_1,-23.53,27.47,2019,1,0.000077,0.454468,0.000034,0.114291,-80.286309,...,74897.052000,2180.266538,81287.074479,1562.031017,10.734123,0.280660,-31.230174,33.337966,-93.217261,23.358251
1,ID_-23.53_27.47_2019_2,-23.53,27.47,2019,2,0.000181,0.423171,0.000062,0.075100,-17.765690,...,84222.653521,982.619057,90307.650011,927.111222,5.250020,0.238933,66.490898,58.137794,-85.001467,13.416481


In [9]:
train.columns.difference(test.columns)

Index(['emission'], dtype='object')

# Splitting data

In [10]:
features = train.select_dtypes(["int", "float"]).columns.intersection(test.select_dtypes(["int", "float"]).columns).tolist()
features

['latitude',
 'longitude',
 'year',
 'week_no',
 'SulphurDioxide_SO2_column_number_density',
 'SulphurDioxide_SO2_column_number_density_amf',
 'SulphurDioxide_SO2_slant_column_number_density',
 'SulphurDioxide_cloud_fraction',
 'SulphurDioxide_sensor_azimuth_angle',
 'SulphurDioxide_sensor_zenith_angle',
 'SulphurDioxide_solar_azimuth_angle',
 'SulphurDioxide_solar_zenith_angle',
 'SulphurDioxide_SO2_column_number_density_15km',
 'CarbonMonoxide_CO_column_number_density',
 'CarbonMonoxide_H2O_column_number_density',
 'CarbonMonoxide_cloud_height',
 'CarbonMonoxide_sensor_altitude',
 'CarbonMonoxide_sensor_azimuth_angle',
 'CarbonMonoxide_sensor_zenith_angle',
 'CarbonMonoxide_solar_azimuth_angle',
 'CarbonMonoxide_solar_zenith_angle',
 'NitrogenDioxide_NO2_column_number_density',
 'NitrogenDioxide_tropospheric_NO2_column_number_density',
 'NitrogenDioxide_stratospheric_NO2_column_number_density',
 'NitrogenDioxide_NO2_slant_column_number_density',
 'NitrogenDioxide_tropopause_pressure'

In [11]:
train[features].var()[train[features].var()<5e-8]

SulphurDioxide_SO2_slant_column_number_density             3.946017e-08
SulphurDioxide_SO2_column_number_density_15km              6.153494e-09
NitrogenDioxide_NO2_column_number_density                  2.386937e-09
NitrogenDioxide_tropospheric_NO2_column_number_density     2.467667e-09
NitrogenDioxide_stratospheric_NO2_column_number_density    3.879787e-11
NitrogenDioxide_NO2_slant_column_number_density            3.654368e-09
Formaldehyde_tropospheric_HCHO_column_number_density       9.329211e-09
Formaldehyde_HCHO_slant_column_number_density              7.980467e-09
dtype: float64

In [12]:
features = list(set(features).difference(train[features].var()[train[features].var()<5e-8].index.tolist()))
features

['Cloud_cloud_base_height',
 'UvAerosolLayerHeight_sensor_azimuth_angle',
 'week_no',
 'Cloud_solar_azimuth_angle',
 'UvAerosolLayerHeight_aerosol_optical_depth',
 'Cloud_cloud_top_pressure',
 'UvAerosolLayerHeight_aerosol_pressure',
 'NitrogenDioxide_cloud_fraction',
 'NitrogenDioxide_sensor_azimuth_angle',
 'Ozone_solar_zenith_angle',
 'Cloud_cloud_fraction',
 'Ozone_sensor_zenith_angle',
 'SulphurDioxide_SO2_column_number_density',
 'Cloud_cloud_base_pressure',
 'Cloud_sensor_zenith_angle',
 'SulphurDioxide_solar_zenith_angle',
 'Cloud_cloud_optical_depth',
 'UvAerosolLayerHeight_solar_zenith_angle',
 'CarbonMonoxide_sensor_azimuth_angle',
 'NitrogenDioxide_absorbing_aerosol_index',
 'latitude',
 'NitrogenDioxide_solar_zenith_angle',
 'Formaldehyde_sensor_azimuth_angle',
 'SulphurDioxide_sensor_azimuth_angle',
 'UvAerosolIndex_sensor_azimuth_angle',
 'Ozone_O3_slant_column_number_density',
 'UvAerosolIndex_absorbing_aerosol_index',
 'Cloud_cloud_top_height',
 'CarbonMonoxide_CO_colu

In [13]:
target = "emission"

In [14]:
X, y = train[features], train[target]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
print("Number of samples in X_train dataset: ", X_train.shape) 
print("Number of samples in y_train dataset: ", y_train.shape) 
print("Number of samples in X_test dataset: ", X_test.shape) 
print("Number of samples in y_test dataset: ", y_test.shape)

Number of samples in X_train dataset:  (59204, 66)
Number of samples in y_train dataset:  (59204,)
Number of samples in X_test dataset:  (14801, 66)
Number of samples in y_test dataset:  (14801,)


In [17]:
train_pool = Pool(
    data=X_train, 
    label=y_train
)

# create validation_pool object
validation_pool = Pool(
    data=X_test, 
    label=y_test
)

# Train model

In [ ]:
# pretty basic model, max_depth=10 give slightly better results
cbs = CatBoostRegressor(iterations=30000,
                         learning_rate=0.0012,
                         loss_function='RMSE',
                         max_depth=10, 
                         early_stopping_rounds=200,
                         task_type = "CPU",
                        #  cat_features = cat_features
                        )

# we are passing categorical features as parameters here
cbs.fit(
    train_pool,
    eval_set=validation_pool,
    verbose=500,
    plot=True 
);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 53106.6718529	test: 50097.4504514	best: 50097.4504514 (0)	total: 279ms	remaining: 2h 19m 19s
500:	learn: 32970.3746741	test: 31344.4383811	best: 31344.4383811 (500)	total: 2m 24s	remaining: 2h 21m 21s
1000:	learn: 21675.5015866	test: 20889.0414275	best: 20889.0414275 (1000)	total: 4m 39s	remaining: 2h 15m 10s
1500:	learn: 14997.5449056	test: 14663.9292905	best: 14663.9292905 (1500)	total: 7m 1s	remaining: 2h 13m 14s
2000:	learn: 11763.3481590	test: 11629.8969373	best: 11629.8969373 (2000)	total: 9m 22s	remaining: 2h 11m 5s
2500:	learn: 10115.5862604	test: 10055.1286934	best: 10055.1286934 (2500)	total: 11m 53s	remaining: 2h 10m 45s
3000:	learn: 9058.0276087	test: 9052.8948940	best: 9052.8948940 (3000)	total: 14m 31s	remaining: 2h 10m 36s
3500:	learn: 8158.8395494	test: 8216.6583087	best: 8216.6583087 (3500)	total: 17m 14s	remaining: 2h 10m 27s
4000:	learn: 7447.8265779	test: 7569.0582808	best: 7569.0582808 (4000)	total: 20m 2s	remaining: 2h 10m 13s
4500:	learn: 6837.2841883	t

# Assess model

In [ ]:
test_predictions = cbs.predict(X_test).flatten()

# error = (test_predictions - y_test)

# plt.figure(figsize=(10,10))
# plt.scatter(y_test, 
#             test_predictions, 
#             c=error,
#             s=1.9,
#             cmap='hsv'
#             )
# plt.colorbar()
# plt.xlabel('True Values [emission]')
# plt.ylabel('Predictions [emission]')
# plt.axis('equal')
# plt.axis('square')
# plt.xlim([0, plt.xlim()[1]])
# plt.ylim([0, plt.ylim()[1]])
# plt.show()

In [ ]:
# error = test_predictions - y_test
# # print(type(error))

# plt.figure(figsize=(10,10))
# plt.scatter(y_test, 
#             test_predictions, 
#             c=error,
#             s=2,
#             cmap='hsv',
#             )
# plt.colorbar()
# plt.xlabel('True Values [emission]')
# plt.ylabel('Predictions [emission]')
# plt.axis('equal')
# plt.axis('square')
# plt.xlim([0, 30000])
# plt.ylim([0, 30000])
# plt.show()

In [ ]:
# plt.figure(figsize=(10,10))
# plt.hist2d(y_test, test_predictions, (500,500),cmap=plt.cm.jet)
# plt.colorbar()
# plt.xlim([0, 30000])
# plt.ylim([0, 30000])
# plt.show()

In [ ]:
# plt.figure(figsize=(16,7))
# plt.hist(error, bins = 300, rwidth=0.9)
# plt.xlabel('Predictions Error [emission]')
# _ = plt.ylabel('Count')
# plt.xlim([-8000, 8000])
# plt.show()

In [ ]:
# %%time

# importance_types = ['PredictionValuesChange',
#                     'LossFunctionChange'
#                    ]


# for importance_type in importance_types:
#     print(importance_type)
#     print(cbs.get_feature_importance(data=train_pool, 
#                                      type=importance_type))
#     print('\n\n\n\n')

In [ ]:
# %%time

# import shap
# shap.initjs()

# shap_values = cbs.get_feature_importance(Pool(X_test, 
#                                               label=y_test,
#                                               # cat_features=cat_features
#                                               ), 
#                                          type="ShapValues")
# print(type(shap_values))

# expected_value = shap_values[0,-1]
# print(expected_value)

# shap_values = shap_values[:,:-1]

In [ ]:
# shap.summary_plot(shap_values, X_test, max_display=X_test.shape[1])

In [ ]:
# shap.dependence_plot(ind='longitude', 
#                      interaction_index='longitude',
#                      shap_values=shap_values, 
#                      features=X_test,  
#                      display_features=X_test)

In [ ]:
# shap.dependence_plot(ind='latitude', 
#                      interaction_index='latitude',
#                      shap_values=shap_values, 
#                      features=X_test,  
#                      display_features=X_test)

In [ ]:
# shap.initjs()
# shap.force_plot(expected_value, shap_values[:1000,:], X_test.iloc[:1000,:])

In [ ]:
# shap.initjs()
# for i in range(20,30):
#     print('Sample', i, 'from the test set:')
#     display(shap.force_plot(expected_value, shap_values[i,:], X_test.iloc[i,:]))
#     print('Listed_price -------------------------------------->', y_test.iloc[i])
#     print('parameters:\n', X_test.iloc[i,:])
#     print('\n\n\n\n\n\n\n')

# Submission

In [ ]:
ss["emission"] = cbs.predict(test[features])

In [ ]:
ss.to_csv("../Submissions/sub3_cpu.csv", index=False)

In [ ]:
from google.colab import files

In [ ]:
files.download("../Submissions/sub3_cpu.csv")